In [23]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: print("\n".join(map(str, path.iterdir())))

sys.path.append('../src')
mturk_path = Path("../mturk/results")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from instruction_parsing import program, Commands
import annotation_io
import read_data
annotation_io.set_recipe_source("npn-cooking")
from edit_distance import edit_distance


def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

def annotator_agreement(annotation1,annotation2, ignore_commands=[]):
    lst1 = [(x.command.name,x.arg,x.resource) for x in program(annotation1) if x.command not in ignore_commands]
    lst2 = [(x.command.name,x.arg,x.resource) for x in program(annotation2) if x.command not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))


In [25]:
ls(mturk_path)

../mturk/results/4335716.json
../mturk/results/Batch_4430799_batch_results.csv
../mturk/results/Batch_4430799_batch_results.csv:Zone.Identifier
../mturk/results/Batch_4438769_batch_results.csv:Zone.Identifier
../mturk/results/results_approved.csv
../mturk/results/4289268.json
../mturk/results/Batch_4258572_batch_results.csv
../mturk/results/Batch_4286064_batch_results.csv
../mturk/results/Batch_4438769_batch_results.csv
../mturk/results/Batch_4301520_batch_results.csv
../mturk/results/Batch_4289268_batch_results.csv
../mturk/results/Batch_4280306_batch_results.csv
../mturk/results/4286064.json
../mturk/results/4301520.json
../mturk/results/Batch_4335716_batch_results.csv


In [36]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    #return df
    #hack:
    df = df.rename(columns={"Input.v11":"Answer.Title", "Input.v3": "Answer.ID"})
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    df = df.drop("id", axis=1).rename(columns={"ID": "id"})
    return df.set_index("AssignmentId")
# batch = 4430799 # 20
batch = 4438769 # 93
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,id,Title,actions,ingredients,msgbox_count,n_actions,seconds_spent
AssignmentId,,,,,,,,,,
39L1G8WVWQSZJ4NTJMHVFB47WVN31M,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A25UMKJS7PC293,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IRghrxgvV...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",3,17,"[39,12,60,53,16,60,9,26,86]"
3B2X28YI3WGZTLS34L8ISQT0SYIB6L,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AM4PAC2YQPREB,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IRghrxgvV...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",3,21,"[96,26,125,86,29,57,27,76,86]"
3WI0P0II61TLU1O7A52MYB8WR7ERDE,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A2ZJAQE1IFJRZF,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":14,""command"":""0"",""ts"":2,""arg"":""undefine...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",7,18,"[63,3,102,34,23,115,22,41,125]"
3VP0C6EFSGXVCCW0VAYR664U4XH6M0,3HUR21WDDUQ53O7GPYQQCV9UMSVYX3,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A12EO8AAUDZIU8,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":2,""command"":""0"",""ts"":1,""arg"":""IaGLvtAKO...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""cr...",2,24,"[245,223,141,278,100,55,78,139,22]"
3WS1NTTKEYDUSRI47YEX2FAUUTF0FI,3HUR21WDDUQ53O7GPYQQCV9UMSVYX3,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AYC1FUXP2CV5G,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":2,""command"":""0"",""ts"":0,""arg"":""IaGLvtAKO...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""ch...",2,17,"[20,54,32,236,32,13,19,27,46]"
...,...,...,...,...,...,...,...,...,...,...
3E1QT0TDFPAWK7PX3AXXYRT4TJOI8J,3YZ7A3YHR5UZ5JBXKMF62QU9C8Q5SN,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A31PVZ4KDUMEEV,mushrooms_montreal,Mushrooms Montreal,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""Iz0wiMjVJ...","[{""name"":""cream"",""id"":""IztU6mhCZ""},{""name"":""le...",0,13,"[65,54,44,28,103,3,4,3,18]"
3QJOXOW4XJSZPRPF1SHPIUW7LV6EMU,3YZ7A3YHR5UZ5JBXKMF62QU9C8Q5SN,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AM4PAC2YQPREB,mushrooms_montreal,Mushrooms Montreal,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IztU6mhCZ...","[{""name"":""cream"",""id"":""IztU6mhCZ""},{""name"":""le...",1,10,"[27,55,59,46,59,6,4,2,11]"
36PW28KO4ZXYNQGQ4PRK9317REGAE5,3EHIMLB7F70T2SGJ7QAAAWXY5NA8H7,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A28QUR0QYD2WI7,suppers_[tuesday]_carrot__ham_soup,Suppers [Tuesday] Carrot Ham Soup,"[{""id"":0,""command"":""2"",""ts"":1,""arg"":""TPAN"",""ar...","[{""name"":""butter"",""id"":""I10_d4pRP""},{""name"":""c...",1,16,"[60,96,54,11,12,50,24,29]"


In [57]:
def actions2program(actions):
    cmds = ["PUT", "REMOVE", "USE", "STOP_USING","CHEF_CHECK","DO","MOVE_CONTENTS"]
    prog = []
    if type(actions)==str:
        actions = json.loads(actions)
    for a in actions:
        prog.append((int(a["ts"]), cmds[int(a["command"])], a["arg"], a["resource"]))
    prog = sorted(prog)
    prog = [c[1:] for c in prog]
    return prog
df["program"] = df["actions"].apply(actions2program)
df["n"] = df["program"].apply(len)
df[["WorkerId","id", "n"]]

,WorkerId,id,n
AssignmentId,,,
39L1G8WVWQSZJ4NTJMHVFB47WVN31M,A25UMKJS7PC293,cuban_red_beans__rice,17
3B2X28YI3WGZTLS34L8ISQT0SYIB6L,AM4PAC2YQPREB,cuban_red_beans__rice,21
3WI0P0II61TLU1O7A52MYB8WR7ERDE,A2ZJAQE1IFJRZF,cuban_red_beans__rice,18
3VP0C6EFSGXVCCW0VAYR664U4XH6M0,A12EO8AAUDZIU8,augustas_chilled_tomato_soup_basil_cream,24
3WS1NTTKEYDUSRI47YEX2FAUUTF0FI,AYC1FUXP2CV5G,augustas_chilled_tomato_soup_basil_cream,17
...,...,...,...
3E1QT0TDFPAWK7PX3AXXYRT4TJOI8J,A31PVZ4KDUMEEV,mushrooms_montreal,13
3QJOXOW4XJSZPRPF1SHPIUW7LV6EMU,AM4PAC2YQPREB,mushrooms_montreal,10
36PW28KO4ZXYNQGQ4PRK9317REGAE5,A28QUR0QYD2WI7,suppers_[tuesday]_carrot__ham_soup,16


In [58]:
df#.loc["3DZQRBDBSRAR4M5SWQ7ZVSOZVJ6S3S","program"]

,HITId,HITTypeId,WorkerId,id,Title,actions,ingredients,msgbox_count,n_actions,seconds_spent,program,n,total_seconds
AssignmentId,,,,,,,,,,,,,
39L1G8WVWQSZJ4NTJMHVFB47WVN31M,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A25UMKJS7PC293,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IRghrxgvV...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",3,17,"[39,12,60,53,16,60,9,26,86]","[(PUT, IRghrxgvV, STOVE_MED), (PUT, Is_lS1dmt,...",17,361
3B2X28YI3WGZTLS34L8ISQT0SYIB6L,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AM4PAC2YQPREB,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IRghrxgvV...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",3,21,"[96,26,125,86,29,57,27,76,86]","[(PUT, IRghrxgvV, STOVE_MEDLOW), (PUT, Is_lS1d...",21,608
3WI0P0II61TLU1O7A52MYB8WR7ERDE,34F34TZU7W0E7J9KF94UCI0AWFLJ2J,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A2ZJAQE1IFJRZF,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":14,""command"":""0"",""ts"":2,""arg"":""undefine...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...",7,18,"[63,3,102,34,23,115,22,41,125]","[(PUT, undefined, COUNTER2), (PUT, I-dhcRjk3, ...",18,528
3VP0C6EFSGXVCCW0VAYR664U4XH6M0,3HUR21WDDUQ53O7GPYQQCV9UMSVYX3,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A12EO8AAUDZIU8,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":2,""command"":""0"",""ts"":1,""arg"":""IaGLvtAKO...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""cr...",2,24,"[245,223,141,278,100,55,78,139,22]","[(MOVE_CONTENTS, APPL_SLICE, COUNTER1), (MOVE_...",24,1281
3WS1NTTKEYDUSRI47YEX2FAUUTF0FI,3HUR21WDDUQ53O7GPYQQCV9UMSVYX3,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AYC1FUXP2CV5G,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":2,""command"":""0"",""ts"":0,""arg"":""IaGLvtAKO...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""ch...",2,17,"[20,54,32,236,32,13,19,27,46]","[(PUT, IaGLvtAKO, COUNTER1), (USE, TPAN, COUNT...",17,479
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3E1QT0TDFPAWK7PX3AXXYRT4TJOI8J,3YZ7A3YHR5UZ5JBXKMF62QU9C8Q5SN,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A31PVZ4KDUMEEV,mushrooms_montreal,Mushrooms Montreal,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""Iz0wiMjVJ...","[{""name"":""cream"",""id"":""IztU6mhCZ""},{""name"":""le...",0,13,"[65,54,44,28,103,3,4,3,18]","[(PUT, Iz0wiMjVJ, APPL_SLICE), (CHEF_CHECK, LS...",13,322
3QJOXOW4XJSZPRPF1SHPIUW7LV6EMU,3YZ7A3YHR5UZ5JBXKMF62QU9C8Q5SN,3DBJM5HJP5B5I560ONUA3GMRZP79P6,AM4PAC2YQPREB,mushrooms_montreal,Mushrooms Montreal,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IztU6mhCZ...","[{""name"":""cream"",""id"":""IztU6mhCZ""},{""name"":""le...",1,10,"[27,55,59,46,59,6,4,2,11]","[(PUT, IrXTUISnn, VALID_MIX1), (PUT, IzDRSSnRu...",10,269
36PW28KO4ZXYNQGQ4PRK9317REGAE5,3EHIMLB7F70T2SGJ7QAAAWXY5NA8H7,3DBJM5HJP5B5I560ONUA3GMRZP79P6,A28QUR0QYD2WI7,suppers_[tuesday]_carrot__ham_soup,Suppers [Tuesday] Carrot Ham Soup,"[{""id"":0,""command"":""2"",""ts"":1,""arg"":""TPAN"",""ar...","[{""name"":""butter"",""id"":""I10_d4pRP""},{""name"":""c...",1,16,"[60,96,54,11,12,50,24,29]","[(CHEF_CHECK, LMELT, STOVE_MED), (PUT, I10_d4p...",16,336


# Measure agreement

In [53]:
def annotator_agreement(lst1,lst2, ignore_commands=[],config=False):
    if not config:
        lst1 = [(x[0],x[1],x[2].rsplit('_',1)[0]) for x in lst1]
        lst2 = [(x[0],x[1],x[2].rsplit('_',1)[0]) for x in lst2]
    lst1 = [x for x in lst1 if x[0] not in ignore_commands]
    lst2 = [x for x in lst2 if x[0] not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))

In [54]:
ignore_commands={"CHEF_CHECK","USE","STOP_USING"}
def couple_agreement(lst):
    ret=[]
    for l1, l2 in itertools.combinations(lst,2):
        ret.append(annotator_agreement(l1,l2, ignore_commands))
    return ret
df_agreement=df[df["n"]>0].groupby("id")["program"].apply(list).apply(couple_agreement)
df_agreement

id
all_in_one_casserole                                       [0.5384615384615384, 0.4736842105263158, 0.478...
au_gratin_beef_spinach                                     [0.6363636363636364, 0.5238095238095238, 0.476...
augustas_chilled_tomato_soup_basil_cream                                [0.46875, 0.5277777777777778, 0.625]
baked_chicken_tomato_sauce                                     [0.9565217391304348, 0.5, 0.4782608695652174]
baked_zucchini_tomatoes                                    [0.48484848484848486, 0.26086956521739135, 0.2...
                                                                                 ...                        
weber_tangy_barbeque_sauce                                 [0.8636363636363636, 0.8636363636363636, 0.954...
wintery_day_bean_soup                                                        [0.6, 0.7083333333333333, 0.56]
zucchini-corn_skillet                                      [0.9411764705882353, 0.9473684210526316, 0.888...
zucchini_potatoe

In [55]:
df_agreement.apply(np.mean)

id
all_in_one_casserole                                       0.496802
au_gratin_beef_spinach                                     0.545455
augustas_chilled_tomato_soup_basil_cream                   0.540509
baked_chicken_tomato_sauce                                 0.644928
baked_zucchini_tomatoes                                    0.339482
                                                             ...   
weber_tangy_barbeque_sauce                                 0.893939
wintery_day_bean_soup                                      0.622778
zucchini-corn_skillet                                      0.925811
zucchini_potatoes                                          0.511156
zuppa_di_riso_e_pecorino_romano_ricepecorino_romano_sou    0.746032
Name: program, Length: 93, dtype: float64

In [56]:
df_agreement.apply(np.mean).mean()

0.6394030325483624

# Measure time

In [61]:
df["total_seconds"]=df["seconds_spent"].apply(json.loads).apply(sum)
df[df["n"]>0].groupby("id")["total_seconds"].mean().mean()

519.362007168459